In [64]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [65]:
odds_f = pd.read_csv(r"E:\project\nba_odds_study\data\odds\bettingdata_com\processed\bettingdata_odds.csv", parse_dates=["datetime"], index_col="datetime")

In [66]:
pre_f = pd.read_csv(r"predict_data_with_combined.csv")

In [67]:
odds_f.columns

Index(['Season', 'Type', 'DayString', 'TimeString', 'DayOfWeek',
       'DateTimeString', 'LastUpdated', 'Status', 'AwayTeam', 'AwayTeamScore',
       ...
       'BarstoolAwayPointSpread', 'BarstoolHomePointSpreadPayout',
       'BarstoolAwayPointSpreadPayout', 'BarstoolOverUnder',
       'BarstoolOverPayout', 'BarstoolUnderPayout', 'BarstoolHomeTeamTotal',
       'BarstoolAwayTeamTotal', 'BarstoolHomeTeamTotalPayout',
       'BarstoolAwayTeamTotalPayout'],
      dtype='object', length=143)

In [68]:
pre_f.columns

Index(['Unnamed: 0', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA',
       'FG_PCT',
       ...
       'BLK_RANK.1', 'BLKA_RANK.1', 'PF_RANK.1', 'PFD_RANK.1', 'PTS_RANK.1',
       'PLUS_MINUS_RANK.1', 'Home-Team-Win', 'date', 'winprob_home',
       'winprob_away'],
      dtype='object', length=112)

In [69]:
odds_f.head(5)

,Season,Type,DayString,TimeString,DayOfWeek,DateTimeString,LastUpdated,Status,AwayTeam,AwayTeamScore,...,BarstoolAwayPointSpread,BarstoolHomePointSpreadPayout,BarstoolAwayPointSpreadPayout,BarstoolOverUnder,BarstoolOverPayout,BarstoolUnderPayout,BarstoolHomeTeamTotal,BarstoolAwayTeamTotal,BarstoolHomeTeamTotalPayout,BarstoolAwayTeamTotalPayout
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-02,2019,RegularSeason,1/2/19,8:00 PM ET,Wed,1/2/19 @ 8:00 PM ET,NaN,Final,ORL,112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,2019,RegularSeason,1/2/19,10:30 PM ET,Wed,1/2/19 @ 10:30 PM ET,NaN,Final,OKC,107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,2019,RegularSeason,1/2/19,9:00 PM ET,Wed,1/2/19 @ 9:00 PM ET,NaN,Final,PHI,132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,2019,RegularSeason,1/2/19,8:00 PM ET,Wed,1/2/19 @ 8:00 PM ET,NaN,Final,DET,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,2019,RegularSeason,1/2/19,8:00 PM ET,Wed,1/2/19 @ 8:00 PM ET,NaN,Final,MIN,102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
odds_f.drop(datetime(2019, 2, 17), axis=0, inplace=True)
odds_f.drop(datetime(2022, 2, 20), axis=0, inplace=True)
odds_f.drop(datetime(2020, 2, 16), axis=0, inplace=True)
odds_f.drop(datetime(2021, 3, 7), axis=0, inplace=True)

In [71]:
odds_f_sub = odds_f[["AwayTeam", "HomeTeam","HomeTeamHasWon","ConsensusHomeMoneyLine", "ConsensusAwayMoneyLine"]].copy()

In [72]:
odds_f_sub.head()

,AwayTeam,HomeTeam,HomeTeamHasWon,ConsensusHomeMoneyLine,ConsensusAwayMoneyLine
datetime,,,,,
2019-01-02,ORL,CHI,False,102.0,-112.0
2019-01-02,OKC,LAL,False,190.0,-210.0
2019-01-02,PHI,PHO,False,150.0,-160.0
2019-01-02,DET,MEM,False,-240.0,220.0
2019-01-02,MIN,BOS,True,-250.0,230.0


In [73]:
pre_f.head()

,Unnamed: 0,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,BLK_RANK.1,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Home-Team-Win,date,winprob_home,winprob_away
0,0,Charlotte Bobcats,9,5,4,0.556,48.0,33.2,78.0,0.426,...,28,2,6,15,25,21,1,2007-11-19,0.638390,0.361610
1,1,Memphis Grizzlies,9,2,7,0.222,48.6,37.0,80.3,0.461,...,14,17,26,23,13,27,1,2007-11-19,0.667728,0.332272
2,2,New Orleans Hornets,11,9,2,0.818,48.5,37.9,81.7,0.464,...,25,5,5,2,8,6,0,2007-11-19,0.589039,0.410961
3,3,Utah Jazz,11,7,4,0.636,48.0,40.0,82.9,0.482,...,4,11,27,3,30,27,1,2007-11-19,0.760183,0.239817
4,0,Washington Wizards,9,4,5,0.444,48.6,34.3,83.3,0.412,...,13,8,16,23,27,19,1,2007-11-20,0.667034,0.332966


In [74]:
pre_f.loc[:, "datetime"]  = pre_f.loc[:, "date"]

In [75]:
pre_f.head()

,Unnamed: 0,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Home-Team-Win,date,winprob_home,winprob_away,datetime
0,0,Charlotte Bobcats,9,5,4,0.556,48.0,33.2,78.0,0.426,...,2,6,15,25,21,1,2007-11-19,0.638390,0.361610,2007-11-19
1,1,Memphis Grizzlies,9,2,7,0.222,48.6,37.0,80.3,0.461,...,17,26,23,13,27,1,2007-11-19,0.667728,0.332272,2007-11-19
2,2,New Orleans Hornets,11,9,2,0.818,48.5,37.9,81.7,0.464,...,5,5,2,8,6,0,2007-11-19,0.589039,0.410961,2007-11-19
3,3,Utah Jazz,11,7,4,0.636,48.0,40.0,82.9,0.482,...,11,27,3,30,27,1,2007-11-19,0.760183,0.239817,2007-11-19
4,0,Washington Wizards,9,4,5,0.444,48.6,34.3,83.3,0.412,...,8,16,23,27,19,1,2007-11-20,0.667034,0.332966,2007-11-20


In [76]:
pre_f.rename(columns={"winprob_away": "away_predict_winprob",
                   "winprob_home": "home_predict_winprob",
                   "TEAM_NAME": "home_team",
                   "TEAM_NAME.1": "away_team"}, inplace=True)

In [77]:
odds_f_sub.columns

Index(['AwayTeam', 'HomeTeam', 'HomeTeamHasWon', 'ConsensusHomeMoneyLine',
       'ConsensusAwayMoneyLine'],
      dtype='object')

In [78]:
odds_f_sub.rename(columns = {
    'AwayTeam':'away_team',
    'HomeTeam':'home_team',
    'HomeTeamHasWon':'home_win',
    'ConsensusHomeMoneyLine': 'money_line_home',
    'ConsensusAwayMoneyLine': 'money_line_away'
}, inplace=True)

In [79]:
odds_f_sub.head()

,away_team,home_team,home_win,money_line_home,money_line_away
datetime,,,,,
2019-01-02,ORL,CHI,False,102.0,-112.0
2019-01-02,OKC,LAL,False,190.0,-210.0
2019-01-02,PHI,PHO,False,150.0,-160.0
2019-01-02,DET,MEM,False,-240.0,220.0
2019-01-02,MIN,BOS,True,-250.0,230.0


In [80]:
odds_f_sub["away_team"].unique()

array(['ORL', 'OKC', 'PHI', 'DET', 'MIN', 'NO', 'DAL', 'ATL', 'MIA',
       'HOU', 'DEN', 'TOR', 'UTA', 'IND', 'WAS', 'BKN', 'LAC', 'NY', 'GS',
       'CHA', 'MEM', 'LAL', 'SA', 'SAC', 'CLE', 'MIL', 'CHI', 'PHO',
       'BOS', 'POR'], dtype=object)

In [81]:
pre_f["away_team"].unique()

array(['Portland Trail Blazers', 'Seattle SuperSonics', 'Orlando Magic',
       'New Jersey Nets', 'Philadelphia 76ers', 'San Antonio Spurs',
       'Milwaukee Bucks', 'Los Angeles Lakers', 'Golden State Warriors',
       'Toronto Raptors', 'Chicago Bulls', 'Phoenix Suns',
       'Washington Wizards', 'Atlanta Hawks', 'Dallas Mavericks',
       'New York Knicks', 'Indiana Pacers', 'Cleveland Cavaliers',
       'Sacramento Kings', 'Denver Nuggets', 'Charlotte Bobcats',
       'Houston Rockets', 'Memphis Grizzlies', 'Minnesota Timberwolves',
       'New Orleans Hornets', 'Los Angeles Clippers', 'Boston Celtics',
       'Miami Heat', 'Utah Jazz', 'Detroit Pistons',
       'Oklahoma City Thunder', 'Brooklyn Nets', 'New Orleans Pelicans',
       'Charlotte Hornets', 'LA Clippers'], dtype=object)

In [82]:
TEAM_NAME_MAPPING = {
 'Atlanta Hawks': 'ATL',
 'Brooklyn Nets': 'BKN',
 'Boston Celtics': 'BOS',
 'Charlotte Hornets': 'CHA',
 'Chicago Bulls': 'CHI',
 'Cleveland Cavaliers': 'CLE',
 'Dallas Mavericks': 'DAL',
 'Denver Nuggets': 'DEN',
 'Detroit Pistons': 'DET',
 'Golden State Warriors': 'GS',
 'Houston Rockets': 'HOU',
 'Indiana Pacers': 'IND',
 'Los Angeles Clippers': 'LAC',
 'Los Angeles Lakers': 'LAL',
 'Memphis Grizzlies': 'MEM',
 'Miami Heat': 'MIA',
 'Milwaukee Bucks': 'MIL',
 'Minnesota Timberwolves': 'MIN',
 'New Orleans Pelicans': 'NO',
 'New York Knicks': 'NY',
 'Oklahoma City Thunder': 'OKC',
 'Orlando Magic': 'ORL',
 'Philadelphia 76ers': 'PHI',
 'Phoenix Suns': 'PHO',
 'Portland Trail Blazers': 'POR',
 'Sacramento Kings': 'SAC',
 'San Antonio Spurs': 'SA',
 'Toronto Raptors': 'TOR',
 'Utah Jazz': 'UTA',
 'Washington Wizards': 'WAS',
 'New Jersey Nets': 'NKN',
 'Seattle SuperSonics': 'SS',
 'Charlotte Bobcats': 'CB',
 'New Orleans Hornets': 'NOH',
 'LA Clippers': 'LC',
}

In [83]:
len(pre_f["away_team"].unique())

35

In [84]:
pre_f["away_team"] = pre_f["away_team"].apply(lambda x: TEAM_NAME_MAPPING[x])
pre_f["home_team"] = pre_f["home_team"].apply(lambda x: TEAM_NAME_MAPPING[x])

In [85]:
pre_f["away_team"].unique()

array(['POR', 'SS', 'ORL', 'NKN', 'PHI', 'SA', 'MIL', 'LAL', 'GS', 'TOR',
       'CHI', 'PHO', 'WAS', 'ATL', 'DAL', 'NY', 'IND', 'CLE', 'SAC',
       'DEN', 'CB', 'HOU', 'MEM', 'MIN', 'NOH', 'LAC', 'BOS', 'MIA',
       'UTA', 'DET', 'OKC', 'BKN', 'NO', 'CHA', 'LC'], dtype=object)

In [86]:
odds_f_sub

,away_team,home_team,home_win,money_line_home,money_line_away
datetime,,,,,
2019-01-02,ORL,CHI,False,102.0,-112.0
2019-01-02,OKC,LAL,False,190.0,-210.0
2019-01-02,PHI,PHO,False,150.0,-160.0
2019-01-02,DET,MEM,False,-240.0,220.0
2019-01-02,MIN,BOS,True,-250.0,230.0
...,...,...,...,...,...
2022-03-27,PHI,PHO,True,-185.0,152.0
2022-03-27,GS,WAS,True,226.0,-281.0
2022-03-27,LAL,NO,True,-173.0,143.0


In [87]:
odds_f_sub.loc[:, "key"] = odds_f_sub.index.astype(str) + odds_f_sub["home_team"] + odds_f_sub["away_team"]
pre_f.loc[:, "key"] = pre_f["datetime"].astype(str) + pre_f["home_team"] + pre_f["away_team"]

In [88]:
odds_f_sub.set_index("key", inplace = True)
pre_f.set_index("key", inplace = True)

In [89]:
odds_f_sub.merge(pre_f, left_index=True, right_index=True, how='inner').to_csv("data.csv")

In [90]:
combine_f = odds_f_sub.merge(pre_f, left_index=True, right_index=True, how='inner')

In [91]:
(combine_f.home_team_x == combine_f.home_team_y).all()

True

In [92]:
combine_f.columns

Index(['away_team_x', 'home_team_x', 'home_win', 'money_line_home',
       'money_line_away', 'Unnamed: 0', 'home_team_y', 'GP', 'W', 'L',
       ...
       'BLKA_RANK.1', 'PF_RANK.1', 'PFD_RANK.1', 'PTS_RANK.1',
       'PLUS_MINUS_RANK.1', 'Home-Team-Win', 'date', 'home_predict_winprob',
       'away_predict_winprob', 'datetime'],
      dtype='object', length=118)

In [93]:
(combine_f["home_win"] == combine_f["Home-Team-Win"])

key
2019-01-02CHIORL    True
2019-01-02LALOKC    True
2019-01-02PHOPHI    True
2019-01-02MEMDET    True
2019-01-02BOSMIN    True
                    ... 
2022-03-27PHOPHI    True
2022-03-27WASGS     True
2022-03-27NOLAL     True
2022-03-27DALUTA    True
2022-03-27BKNCHA    True
Length: 3263, dtype: bool